## Checkpoints
- [ ] ``load -> save -> compare``
- [ ] ``make pkg from scratch``
- [ ] ``layers generator``

In [9]:
import os

conda_env = bool(os.getenv('CONDA_DEFAULT_ENV')) or bool(os.getenv('CONDA_PYTHON_EXE'))  or bool(os.getenv('CONDA_DIR'))
print(conda_env)

True


In [10]:
upgrade_py = False
if upgrade_py:
    if conda_env:
        !conda install -y python-dotenv
    else:
        !python3.10 -m pip install --upgrade pip
        !pip install graph-notebook
        !pip install graphviz
        !pip install pydot
        !pip install networkx
        !pip install pyvis
        !pip install platform
        !pip install hashlib
        !pip install jycm
        !pip install pipreqs
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


ERROR: Could not find a version that satisfies the requirement platform (from versions: none)
ERROR: No matching distribution found for platform
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [11]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import json
import pprint
import numpy as np
import pandas as pd
import time
import platform
import hashlib
from enum import Enum
from pyvis.network import Network
from IPython.display import display, HTML
import copy
import tqdm
from jycm.jycm import YouchamaJsonDiffer
from IPython.display import JSON
from IPython.display import Markdown

from pathlib import Path
os.environ["MGENN_DEBUG"] = "Y"


In [13]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())

import mcore as mc

from common import *

In [14]:
pkg = mc.Package()
pkg.loadFile('data/namespacetest_rev0.pkg')
pkg.counts()


'inputs:9, outputs:10, neurons:10, links:10, total:39'

In [17]:
PackageUIHelper.showGraph(pkg, link_to_zero = False)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
{'attenuationPerTick': '0.020000', 'events': [], 'id': 10, 'length': 2, 'receiverId': 64}
[2025-03-28 18:28:45]in[linkEnds.456] link 10 is hanging down
link  0  to  64  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 11, 'length': 2, 'receiverId': 65}
link  Alias1  to  65  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 12, 'length': 2, 'receiverId': 66}
link  Alias2  to  66  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 13, 'length': 2, 'receiverId': 67}
link  Alias3  to  67  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 14, 'length': 2, 'receiverId': 68}
link  Alias4  to  68  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 15, 'length': 2, 'receiverId': 69}
link  Alias5  to  69  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 16, 'length': 2, 'receiverId': 70}
link  Alias6  to  70  sz: 2
{'attenuationPerTick': '0.020000',

## low-level generators

In [ ]:


p = pkg.clone()
n1 = ObjectFactory.makeNeuronData(peak = 10.0, receivers = [], leak = 0.1, currentEnergy = 3.0, id = p.nextId())
p.neurons.append(n1)
p.connect(n1["id"], 64)
p.connect(n1["id"], 65)
p.connect(n1["id"], 69)
g = showGraph(p)
print("new object",n1["id"])
g
pkg = None

## high-level generators

In [ ]:
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
print(f"i_{i1} --[{l1}]-> N_{n1} --[{l2}]-> o_{o1}")
showGraph(pkg)

# simple network

In [ ]:
F.make_quiet()
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
ce = mc.Engine()
ce.core = mc.Core()
ce.core.load(pkg)
ce.tick_offset = 0

df_expected = pd.DataFrame([0.0]*7 + [5.0] + [0.0]*4, columns=['o1_exp'])
df_in = pd.DataFrame([3,0] * 6, columns=['i1'])
out = [0] * 9 + [5, 0]
ticks = 12
df_out = pd.DataFrame()
for t in range(ticks):
    df_out = pd.concat([df_out, ce.run_once(df_in)], ignore_index=True)



In [ ]:
io = pd.concat([df_in, df_out, df_expected], axis=1)
io['error'] = io['o1'].astype(float) - io['o1_exp'].astype(float)
io

# Worm with breakpoints
## make worm

In [ ]:
F.make_quiet()
#F.make_verbose()
pkg = mc.Package.make_empty()
o_end = pkg.new_output("o_end")
w_size = 10
last_o = o_end
last_n = None
ns = []
for i in range(w_size):
    ni = pkg.new_neuron(leak=0.0, peak=5.0, receivers=[])
    pkg.new_link_between(apt=0.0, length=1, src=ni, dst=last_o)
    last_o = pkg.new_output(f"o_end-{i}")
    ns.append(ni)
    if last_n:
        pkg.new_link_between(apt=1.0, length=1, src=ni, dst=last_n)
    last_n = ni
i1 = pkg.new_tape_input("i1", [])
pkg.new_link_between(apt=0.0, length=1, src=i1, dst=last_n)
showGraph(pkg)


## run worm

In [ ]:

ce = None
dirty_pkg = pkg.clone()
ce = mc.Engine()
ce.core = mc.Core()
ce.core.load(dirty_pkg)
ce.tick_offset = 0
ticks = 200
df_expected = pd.DataFrame([1.0]*(20) + [0.0]*(ticks-20), columns=['o_end'])
df_in = pd.DataFrame([1.0]*(20) + [0.0]*(ticks-20), columns=['i1'])
total_energy = []

df_out = pd.DataFrame()
for t in tqdm.tqdm(range(ticks)):
    df_out = pd.concat([df_out, ce.run_once(df_in)], ignore_index=True)
    dpkg = ce.core.dump()
    energy = dpkg.total_energy()
    total_energy.append(energy)
e_df = pd.DataFrame(total_energy, columns=["E"])
ax = e_df.plot(kind='line', title='total E', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()



## bars output
ax = df_out.plot(kind='bar', title='Outputs B', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()

#line output
ax = df_out.plot(kind='line', title='Outputs L', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()

In [ ]:
df_out